## Purpose:

### Merge in draws from the parallelized decomp analysis, create a multidimensional array and spit out whatever is needed with respect to that array

### Created by : Nafis Sadat (04/13/2017)

In [1]:
## Bring in library
# rm(list=ls())

## Parallel environment
require(doParallel)

require(foreign)
require(readstata13)
require(abind)
require(rlist)
require(data.table)

require(parallel)
require(feather)

Loading required package: doParallel
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: foreign
Loading required package: readstata13
Loading required package: abind
Loading required package: rlist
Loading required package: data.table
Loading required package: feather


In [2]:
## Create metadata for one

x = 0
d1<-data.table(read.dta13(paste0("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_",x,".dta")))
d1 <- d1[, draw:= x]

d2 <- melt(d1, id.vars = c("function", "acause", "sex", "age", "draw"), value.name = "data")
colnames(d2) <- c("fn", "acause", "sex", "age", "draw", "variable", "data")
# head(d2)

## Encode to numerics
varnames <- unique(d2[, .(fn, acause, variable)])
varnames <-varnames[, fn_num := as.integer(as.factor(fn))]
varnames <-varnames[, ac_num := as.integer(as.factor(acause))]
varnames <-varnames[, var_num := as.integer(as.factor(variable))]
# head(varnames)

### A version with no variable name
varnames_2 <- unique(varnames[, .(fn, fn_num, acause, ac_num)])
rm(d1)
rm(d2)

In [ ]:
# ## Try to bring in data on parallel
# acomb <- function(...) abind(..., along=6)
    
# system.time(test<-  foreach(x=0:999,  .export=c("varnames_2"), .combine='rbind',
#                             .packages = c("data.table", "readstata13", "foreign")) %do% {
    
#     ## Bring in data
#     tmp<- data.table(read.dta13(paste0("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_", x, ".dta")))[,draw:=x]
    
#     ## Int to save space
#     tmp <- tmp[, age:= as.integer(age)]
#     tmp <- tmp[, sex:= as.integer(sex)]
    
#     ## Rename out function because function
#     colnames(tmp)[4] <- "fn"
    
#     ## Bring draw up front
#     setcolorder(tmp, c("draw", colnames(tmp)[1:dim(tmp)[2]-1]))
    

#     ## Convert to matrix
# #     tmp <- as.matrix(tmp)
    
#     ## Melt
#     tmp <- melt(tmp, id.vars = c("fn", "acause", "draw", "age", "sex"), value.name = "data", variable.name = "variable")
    
#     ## Merge in metadata
#     tmp <- merge(varnames, tmp , by=c("fn", "acause", "variable"))
    
    
#     ## Drop crap
#     tmp <- tmp[, fn:=NULL]
#     tmp <- tmp[, acause:=NULL]
#     tmp <- tmp[, variable:=NULL]
    
#     if(x %% 100 == 0) {
#         print(paste0("Iteration: ", x, ", ", Sys.time()))
#     }
#     return(tmp)
   
#     ## Convert to an array
# #     reshape2::acast(tmp, fn_num ~ ac_num ~ age ~ sex ~ var_num ~ draw , value.var="data")

#     }
           
# )



In [ ]:
## Is mclapply faster?
system.time(test_mc <- mclapply(c(0:999), function(x) {
    
    ## Bring in data
    tmp<- data.table(read.dta13(paste0("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_", x, ".dta")))
    
    ## Int to save space
    tmp <- tmp[, age:= as.integer(age)]
    tmp <- tmp[, sex:= as.integer(sex)]
    
    ## Rename out function because function
    colnames(tmp)[4] <- "fn"
    
    ## Bring draw up front
#     setcolorder(tmp, c("draw", colnames(tmp)[1:dim(tmp)[2]-1]))
    

    ## Convert to matrix
#     tmp <- as.matrix(tmp)
    
    ## Melt
    tmp <- melt(tmp, id.vars = c("fn", "acause", "age", "sex"), value.name = paste0("d_",x), variable.name = "variable")
    
    ## Merge in metadata
    tmp <- merge(varnames, tmp , by=c("fn", "acause", "variable"), all.y=T)
    
    
#     ## Drop crap
    tmp <- tmp[, fn:=NULL]
    tmp <- tmp[, acause:=NULL]
    tmp <- tmp[, variable:=NULL]
    
    if(x %% 100 == 0 & x>0) {
        print(paste0("Draw: ", x, ", ", Sys.time()))
    }
    return(tmp)
   
    ## Convert to an array
#     reshape2::acast(tmp, fn_num ~ ac_num ~ age ~ sex ~ var_num ~ draw , value.var="data")

    },
                                mc.cores=45, mc.preschedule=F))
# system.time(test_mc <- list.stack(test_mc, data.table=T))

In [ ]:
system.time(test_mc_merged<-Reduce(function(x, y) merge(x, y, by=c("fn_num", "ac_num", "var_num", "age", "sex"), all=T), test_mc))
str(test_mc_merged)

In [ ]:
## Bring in output datas create them into encoded vars
# system.time(all_outputs <- lapply(c(0:999), function(x){
   
#     tmp<- data.table(read.dta13(paste0("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_", x, ".dta")))
#     tmp[, draw:= x]
#     return(tmp)
#     }
#                                  )
#            )

# system.time(stacked_data <- list.stack(all_outputs, data.table = T))

## save(list="stacked_data", file = "/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_all_outputs.Rdata")

# Save memory
# rm(all_outputs)

## Alternatively, just load in the Rdata I made previously
# system.time(load("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_all_outputs.Rdata"))

## Here's a melted version
# system.time(load("/ihme/dex/usa/10_resources/13_other/data/output/compiled_results_super_long.Rdata"))

In [ ]:
## Convert strings to numerics, and set a key
mean_output_long <- merge(mean_output_long, varnames, by=c("fn", "acause", "variable"))
mean_output_long <- mean_output_long[, fn:=NULL]
mean_output_long <- mean_output_long[, acause:=NULL]
mean_output_long <- mean_output_long[, variable:=NULL]
setkeyv(test, c("fn_num", "ac_num", "sex","age","var_num","draw"))

In [ ]:
## Set a key
?setkey

In [ ]:
## Convert to array, parallelizing over draws

#### Time it
system.time(mean_output_array_parallel <- lapply(c(0:999), 
                                                   function(x) reshape2::acast(data = mean_output_long[draw==x],
                           formula =  fn_num ~ ac_num ~ sex ~ age ~ var_num ~ draw, value.var="data")))
    
    
## Array bind across the draw dimension (draw dim = 6)
mean_output_array_parallel_bind<-abind(mean_output_array_parallel, along = 6)
                                                       
# Check for discrepancy (there's be NAs because we don't have a perfectly rectangular dataset)
str(mean_output_array_parallel_bind)    

In [ ]:
## We can collapse in any dimension we want 
## basically, apply the sum over the dimensions we want to keep
## And remove the dimension we want summed over

## e.g. c(2,3,4,5,6) means sum over the function dimension only and keep the rest intact

## Let's sum up all the functions for e.g.
mean_over_fn <- apply(mean_output_array_parallel_bind, c(2,3,4,5,6),  mean, na.rm=T)

In [ ]:
# Orig for debugging purposes
# mean_output_array <- acast(data = mean_output_long,
#                      formula =  fn ~ acause ~ sex ~ age ~ variable , value.var="data")
# str(mean_output_array)

# mean_test <- apply(mean_output_array, c(2,3,4,5), mean, na.rm=T)